# Minilab 3

[[LAB DESCRIPTION HERE]]



In [1]:
import folium
import json
from datascience import *
import pandas as pd

## The datasets
### MTC travel skims
The Metropolitan Transportation Comission (MTC) is the regional transportation planning organization for the Bay Area. They host a database with average travel time, cost, and distance from one traffic analysis zone (TAZ) to all other TAZs in the Bay Area. The files have data for driving alone, car pooling, walking to transit, driving to transit, walking, and biking. 

We have pre-processed the data from the morning commute to include only TAZs around San Francisco, Oakland and Berkeley. The files with inter-TAZ travel time, travel cost, and travel distance are saved with the following filepaths 'data/sf_oak_TimeSkims_AM.csv', 'data/sf_oak_CostSkims_AM.csv' and 'data/sf_oak_DistanceSkims_AM.csv'

More info on the dataset can be found here - http://analytics.mtc.ca.gov/foswiki/Main/SimpleSkims. 
The columns in the data set are below:

    orig	Origin transportation analysis zone	Shape file
    dest	Destination transportation analysis zone	Shape file
    da	    Door-to-door time for the drive alone travel mode (i.e. single occupant private automobile)	 
    daToll	Door-to-door time for the drive alone value toll travel mode (those willing to pay to use an HOT lane)	 
    s2	    Door-to-door time for the shared ride 2 travel mode (i.e. double occupant private automobile)	 
    s2Toll	Door-to-door time for the shared ride 2 value toll travel mode (those willing to pay to use an HOT lane)
    s3	    Door-to-door time for the shared ride 3+ travel mode (i.e. three-or-more occupants traveling in a private vehicle)	 
    s3Toll	Door-to-door time for the shared ride 3+ value toll travel mode (those willing to pay to use an HOT lane, if scenario policy dictates they must pay)	 
    walk	Door-to-door time for walking	 
    bike	Door-to-door time for bicycling	 
    wTrnW	Door-to-door time for walk to transit to walk paths	 
    dTrnW	Door-to-door time for drive to transit to walk paths	 
    wTrnD	Door-to-door time for walk to transit to drive paths (returning home on a park-and-ride tour)

(The raw data with all bay area TAZs can be found at https://mtcdrive.app.box.com/2015-03-116)

### Bay area TAZ geometry data
GeoJSON is a format for encoding a variety of geographic data structures. We have saved a GeoJson file with the Trffic Analysis Zone (TAZ) boundaries for the TAZs around San Francisco, Oakland, and Berkeley. We will use a mapping toolbox called folium to map the TAZs

### Read the data
Normally we use the Table function to read the dataset, but for this lab we are using a mapping tool and we require the data to be in a Pandas Dataframe. The functionality is **very** similar to the table functionality.

In [13]:
# Read the timeskims datafile
data = pd.read_csv('data/sf_oak_TimeSkims_AM.csv')
#print out the first 5 rows of the data table
data.head()

,orig,dest,da,daToll,s2,s2Toll,s3,s3Toll,walk,bike,wTrnW,dTrnW,wTrnD
0,1,1,5.92,5.92,5.92,5.92,5.92,5.92,2.4,0.60,-999.00,-999.00,-999.00
1,1,2,6.25,6.25,6.25,6.25,6.25,6.25,4.8,1.20,7.20,5.75,3.82
2,1,3,6.70,6.70,6.70,6.70,6.70,6.70,8.8,2.20,9.70,5.75,4.42
3,1,4,6.66,6.66,6.66,6.66,6.66,6.66,8.2,2.05,6.12,4.18,3.76
4,1,5,7.34,7.34,7.34,7.34,7.34,7.34,13.6,3.40,7.46,5.25,4.87


In [9]:
# Read the GeoJson file save the data as a string
# json.dumps() dumps the data to a string
geojson_data = json.load(open('data/SF_Oak_TAZs.geojson'))
geojson_str = json.dumps(geojson_data) 

## Mapping the TAZs
We will use a mapping package called Folium to map the TAZs



In [4]:
# Create a folium map object. Set the width, and height of the map image 
# as well as start zoom level and center location
taz_map = folium.Map(width=650, height=500, zoom_start=11,
                     location=[37.8062449,-122.371983])

# Add the taz geojson data layer to the map using the geo_json() method
taz_map.geo_json(geo_str=geojson_str)

# Create the map - notice that a file called map.html has been added to your demos folder 
taz_map.create_map('map.html')

#to show the map in-line
taz_map

## Travel time from one TAZ to all others


In [15]:
#choose an origin TAZ
origin_taz_id=10

# save a new data frame that contains only rows with the origin specified above 
data_origin = data[data['orig']==origin_taz_id]

Folium allows us to color map elemets based on data. We will again use folium's geo_json() method, but now we will use additional arguments to color the TAZs based on travel time. 
We will use the following inputs to the geojson method:

    data: Pandas DataFrame or Series, default None
        Data to bind to the GeoJSON.
    columns: the columns of the dataframe that we will use to color the data
    key_on: string, default None
        Variable in the GeoJSON file to bind the data to. Must always
        start with 'feature' and be in JavaScript objection notation.
        Ex: 'feature.id' or 'feature.properties.statename'. 
    threshold_scale: list, default None
        Data range for D3 threshold scale. Defaults to the following range
        of quantiles: [0, 0.5, 0.75, 0.85, 0.9], rounded to the nearest
        order-of-magnitude integer. Ex: 270 rounds to 200, 5600 to 6000.
    fill_color: string, default 'blue'
        Area fill color. Can pass a hex code, color name, or if you are
        binding data, one of the following color brewer palettes:
        'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu',
        'YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd'.


In [22]:
traveltime_map = folium.Map(width=650, height=500, zoom_start=11,
                            location=[37.8062449,-122.371983])

traveltime_map.geo_json(geo_str=geojson_str, data=data_origin,
                        columns=['dest', 'wTrnW'],
                        threshold_scale=[10,20,30,40,50],
                        key_on='feature.properties.TAZ1454',
                        fill_color='BuPu')

traveltime_map.create_map('map.html')
traveltime_map



In [ ]:
# find origin_taz_id and color it red
for i in range(len(geojson_data['features'])):
    if (geojson_data['features'][i]['properties']['TAZ1454'] == origin_taz_id):
        break

origin_data = geojson_data
origin_data['features']= [origin_data['features'][i]]
origin_str = json.dumps(origin_data)

traveltime_map.geo_json(geo_str=origin_str,fill_color='Red')

traveltime_map.create_map('map.html')
traveltime_map

In [ ]:
# Some questions about travel time

# different modes
# da - notice benefit of proximity to the highway 
# transit - prosimity to bart stops for example


In [ ]:
# now your turn. Let's look at travel costs from one TAZ to all other TAZs

